In [1]:
import os
import dotenv
import guidance

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

gpt4 = guidance.llms("gpt-4")


program = guidance('''
{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}

{{#user~}}
{{utterance}}
{{~/user}}

{{#assistant~}}
{{gen 'response' max_tokens=300}}
{{~/assistant}}
''', llm=gpt4)

program(utterance='Hi there, how are you?')

AttributeError: module 'openai' has no attribute 'OpenAI'

In [11]:
import os
import dotenv
import guidance
from guidance import models, gen, system, user, assistant
from guidance.models import OpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# to use role based chat tags you need a chat model, here we use gpt-3.5-turbo but you can use 'gpt-4' as well
gpt35 = models.OpenAI("gpt-3.5-turbo")

AttributeError: module 'openai' has no attribute 'OpenAI'

In [10]:
from guidance import system, user, assistant

with system():
    lm = gpt35 + "You are a helpful assistant."

with user():
    lm += "What is the meaning of life?"

with assistant():
    lm += gen("response")

AttributeError: module 'openai' has no attribute 'OpenAI'

In [13]:
import guidance

guidance.llms("gpt-4")("What is the meaning of life?")

AttributeError: module 'openai' has no attribute 'OpenAI'